In [14]:
import os
import datetime
import astromatic_wrapper as aw
import astropy.units as u

# Get the current date and time to
log_path = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S_test')

# Set defaults
base_path = '/Users/lasilla/LaSillaRepo/stacking/'
paths = {
    'temp': os.path.join(base_path, 'temp'),
    'log': os.path.join(base_path, 'log', log_path),
    'config': os.path.join(base_path, 'config'),
    'catalogs': os.path.join(base_path, 'catalogs'),
    'stacks': os.path.join(base_path, 'stacks'),
    'images': '/Users/lasilla/dfosc_testdata/reduced/'
}

# If you are using a system wide install you can remove the following code
# that specifies build_paths for each code
build_path = '/Users/lasilla/opt/anaconda2/envs/astromatic'
build_paths = {
    'SExtractor': os.path.join(build_path, 'bin', 'sex'),
    'SCAMP': os.path.join(build_path, 'bin', 'scamp'),
    'SWARP': os.path.join(build_path, 'bin', 'swarp'),
    'PSFEx': os.path.join(build_path, 'bin', 'psfex')
}
# Create the pipeline
pipeline = aw.utils.pipeline.Pipeline(paths, build_paths=build_paths)

/Users/lasilla/opt/anaconda2/envs/astromatic/lib/python2.7/site-packages/astromatic_wrapper/utils/pipeline.py:165: UserWarning: 'log' path has not been set for the pipeline. Log files will not be saved.
  "'log' path has not been set for the pipeline. Log files will not be saved.")


In [15]:
# some properties of the dfosc observations
gain = 0.164 * u.electron / u.adu
ron = 4 * u.electron

In [20]:
# set-up the pipeline
def build_pipeline(pipeline, exposures,
                   ref_catalog='UCAC-4',
                   ref_band = 'DEFAULT',
                   frames = [],
                   stack_name = 'testwasp24.fits',
                   output_cat_name = 'testwasp24.ldac.fits'):
    # Generate catalogs from SExtractor
    catalog_names = []
    for files in exposures:
        # create names for the output catalogs for each image
        catalog_names.append(os.path.join(pipeline.paths['temp'],
                                          os.path.basename(files['image']).replace('.fits',
                                                                                   '.ldac')))
        kwargs  = {
            # image to SExtract
            'files': files,
            'api_kwargs':{
                # arguments to initialize Astromatic class
                'config': {'CATALOG_NAME': catalog_names[-1],
                           'CATALOG_TYPE': 'FITS_LDAC',
                           'DETECT_THRESH':'5.',
                           'ANALYSIS_THRESH':'5.',
                           'FILTER': 'N',
                           'GAIN': '0.164',
                           'ZEROPOINT': '21.',
                           # as the exposures are the "pure counts"
                           'PARAMETERS_NAME':'default.param'},
                # frames to run Sextractor on
                'frames': frames,
                'config_file': os.path.join(pipeline.paths['config'],
                                            'default.sex')
            }
        }
        # Add the step to the pipeline
        pipeline.add_step(aw.api.run_sex, ['step1', 'SExtractor'],
                          ignore_errors=True, ignore_exceptions=True, 
                          ignore_warnings=True,
                          **kwargs)

    # Get astrometric solution from SCAMP
    # Use SCAMP to get astrometric solutions
    kwargs = {
        'catalogs': catalog_names,
        'api_kwargs': {
            'config': {
                'ASTREF_CATALOG': ref_catalog,
                'ASTREF_BAND': ref_band,
                'CATALOG_TYPE': 'FITS_LDAC',
                'SOLVE_PHOTOM': 'N',
                'CHECKPLOT_DEV': 'NULL',
                'NTHREADS':'2'
            },
            'config_file': os.path.join(pipeline.paths['config'],
                                        'default.scamp')


        }
    }
    pipeline.add_step(aw.api.run_scamp, ['step2', 'SCAMP'],**kwargs)

    # Resample (rotate and scale) and combine (stack) images
    stack_filename = os.path.join(pipeline.paths['temp'], stack_name)
    kwargs = {
        'filenames': [exp['image'] for exp in exposures],
        'api_kwargs': {
            'config': {
#                 'WEIGHT_TYPE': 'MAP_WEIGHT',
#                 'WEIGHT_SUFFIX': '.wtmap.fits',
                'IMAGEOUT_NAME': stack_filename,
                'WEIGHTOUT_NAME': stack_filename.replace('.fits','.wtmap.fits'),
                'RESAMPLE_DIR': pipeline.paths['temp'],
            }
        },
        'frames': frames
    }
    pipeline.add_step(aw.api.run_swarp, ['step3', 'SWarp'], **kwargs)

    return pipeline


In [21]:
explist = ['obj_testwasp24_000002.fits', 'obj_testwasp24_000003.fits']

In [22]:
frames = []
exposures = [{'image':os.path.join(pipeline.paths['images'], exposure)} for exposure in explist]
pipeline = build_pipeline(pipeline, exposures=exposures, frames=frames)

In [23]:
print exposures
pipeline.run(['step1'])

[{'image': '/Users/lasilla/dfosc_testdata/reduced/obj_testwasp24_000002.fits'}, {'image': '/Users/lasilla/dfosc_testdata/reduced/obj_testwasp24_000003.fits'}]


PipelineError: Error returned in step 0 (run_step_idx 0)